# Obtención de datos y conección a API

In [1]:
import requests
import pandas as pd

## Selección y extracción de un solo indicador

Selección de un solo país bajo estudio. Si se eligen todos los países la API tarda un poco más de 30 segundos en responder una vez terminado el ciclo for. Y esa carga inicial podría molestar al usuario final.

In [14]:
paises=pd.read_csv('../datasets/paises.csv')

seleccion = input("Ingrese un nombre de país: ")
try:
    pais = paises.loc[paises['country.value'].str.contains(seleccion, case=False),'countryiso3code'].values[0]
except:
    print("No se encontró el país, intente de nuevo")

In [15]:
# Establecemos los parámetros que queremos acceder.
indicador = 'SP.DYN.LE00.IN'
#pais = 'all' # Está comentado porque lo recibe del bloque anterior
iyear = '1980'
fyear = '2022'

# Página de la api y path al recurso solicitado
api_url = ' http://api.worldbank.org/v2/es'
path = f'/country/{pais}/indicator/{indicador}'

url = api_url + path

# Creamos el diccionario con los parametros 
# para el método get
def consultar(pagina = 1):
    args= {
        "date":iyear+":"+fyear,
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
    return requests.get(url,params=args).json()

dict_data = consultar()

In [16]:
# La primera parte de la respuesta nos indica en 
# cuantas páginas de encuentra la información
paginas=dict_data[0]["pages"]
paginas # Retornamos el número de páginas

1

In [17]:
# La segunda parte nos retorna una lista de diccionarios 
# con la información que queríamos
datos=dict_data[1]
datos

[{'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Esperanza de vida al nacer, total (años)'},
  'country': {'id': 'AO', 'value': 'Angola'},
  'countryiso3code': 'AGO',
  'date': '2021',
  'value': None,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Esperanza de vida al nacer, total (años)'},
  'country': {'id': 'AO', 'value': 'Angola'},
  'countryiso3code': 'AGO',
  'date': '2020',
  'value': 61.487,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Esperanza de vida al nacer, total (años)'},
  'country': {'id': 'AO', 'value': 'Angola'},
  'countryiso3code': 'AGO',
  'date': '2019',
  'value': 61.147,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Esperanza de vida al nacer, total (años)'},
  'country': {'id': 'AO', 'value': 'Angola'},
  'countryiso3code': 'AGO',
  'date': '2018',
  'value': 60.782,
  'unit': '',
  '

In [18]:
# Agregamos los valores de las otras páginas, si existen a 
# nuestra lista de diccionarios
for pagina in range(2,paginas+1):
    datos.extend(consultar(pagina)[1])

In [19]:
# Verifico que lo obtenido sea una lista de diccionarios
print(type(datos))
print(type(datos[1]))

<class 'list'>
<class 'dict'>


In [20]:
# Creo el DataFrame con todos los datos
data = pd.json_normalize(datos)

In [21]:
# Visualizo información relevante unicamente
data[['date','value','country.value']]

,date,value,country.value
0,2021,NaN,Angola
1,2020,61.487,Angola
2,2019,61.147,Angola
3,2018,60.782,Angola
4,2017,60.379,Angola
5,2016,59.925,Angola
6,2015,59.398,Angola
7,2014,58.776,Angola
8,2013,58.054,Angola
9,2012,57.236,Angola


In [10]:
# Creo un dataset para trabajar offline
# data.to_csv('../datasets/esperanza_de_vida.csv')

# EDA

Esto se hace en la primera ejecución del código, cuando se corría con todos los países

In [11]:
paises = data[['countryiso3code','country.value']].groupby('countryiso3code').first()

# Regreso el index creado por agrupar a una columna
paises.reset_index(inplace=True)

# Reemplazo todos los valores faltantes por NaN
paises.replace("", float("NaN"), inplace=True)

paises.dropna(inplace=True)

# Creo el dataset de países que me va a servir de selección
# paises.to_csv('../datasets/paises.csv')

Verifico los datos faltantes en la respuesta de la API en español. Son dós nombres de países o grupo de países, con el código AFW y AFE, para Africa ocidental y central y Africa del este y del sur, respectivamente. En resumen: Regiones de Africa. No sé si lo vamos a incluir

In [12]:
data[data.countryiso3code.str.contains('AFE')]

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2021,NaN,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
1,AFE,2020,64.325702,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
2,AFE,2019,64.005213,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
3,AFE,2018,63.648988,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
4,AFE,2017,63.246264,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
5,AFE,2016,62.787681,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
6,AFE,2015,62.259288,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
7,AFE,2014,61.647367,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
8,AFE,2013,60.953363,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,
9,AFE,2012,60.185561,,,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,


## Búsqueda de indicadores

In [13]:
url='http://api.worldbank.org/v2/es/indicator'
def consultar_indicadores(pagina=1):
    argumentos= {
        'page':pagina,
        'per_page':10000,
        'format':'json'
    }
    return requests.get(url,params=argumentos).json()

indicadores = consultar_indicadores()

# la primera parte de la respuesta 
# contiene información sobre las páginas
paginas = indicadores[0]["pages"]

# Reasigno la variable únicamente a los datos de mi interés 
# es una lista de diccionarios
indicadores = indicadores[1]

# recorro todás las páginas que tiene la respuesta 
# y lo voy anexando a la lista principal
for pagina in range(2, paginas+1):
    indicadores.extend(consultar_indicadores(pagina)[1])
indicators = pd.json_normalize(indicadores)

# Dev data para trabajo offline
# indicators.to_csv('../datasets/indicadores.csv')

interes = input("Indicador de interés: ")
indicators.loc[indicators.name.str.contains(interes,case=False),
     ['id','name','sourceNote']]#.to_csv('temp.csv')

,id,name,sourceNote
179,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...
180,3.0.Gini_nozero,Coeficiente de Gini (Ingreso diferente de cero),El coeficiente de Gini es la medida de desigua...
192,3.1.Gini,"Gini, Rural",El coeficiente de Gini es la medida de desigua...
205,3.2.Gini,"Gini, Urbano",El coeficiente de Gini es la medida de desigua...
17088,SI.POV.GINI,Índice de Gini,El índice de Gini mide hasta qué punto la dist...
